## 🎞️ 유사성 점수 계산

추천 시스템 구축 → 데이터 세트 내의 다양한 개체 비교 → `유사성 점수( similarity score )`

점수가 높을수록 개체가 유사하다 !  

1. `유클리드 점수 ( Euclidean Score )` : 두 개체 간의 유클리드 거리가 크다면 유클리드 점수 낮음
2. `피어슨 점수 ( Pearson Score )` : 상관 관계를 측정, 표준편차로 계산 ( -1  ~ 1 )

In [4]:
import argparse
import json
import numpy as np


# 입력 인수를 처리할 인수 파서 -> 사용자 두 명, 점수 계산을 위한 점수 유형 
def build_arg_parser():
    parser = argparse.ArgumentParser(description='Compute similarity score')
    parser.add_argument('--user1', dest='user1', required=True,
            help='First user')
    parser.add_argument('--user2', dest='user2', required=True,
            help='Second user')
    parser.add_argument("--score-type", dest="score_type", required=True, 
            choices=['Euclidean', 'Pearson'], help='Similarity metric to be used')
    return parser


# 사용자 1과 사용자 2 간의 유클리드 거리 점수 계산
def euclidean_score(dataset, user1, user2):
		# 사용자가 데이터 세트에 없으면 오류 발생
    if user1 not in dataset:
        raise TypeError('Cannot find ' + user1 + ' in the dataset')

    if user2 not in dataset:
        raise TypeError('Cannot find ' + user2 + ' in the dataset')

    # 사용자 1과 사용자 2가 평가한 영화 추출
    common_movies = {} 
    for item in dataset[user1]:
        if item in dataset[user2]:
            common_movies[item] = 1

    # 공통으로 평가한 영화가 없다면 유사성 점수 = 0
    if len(common_movies) == 0:
        return 0

		# 두 평점 간의 제곱 차이를 계산
    squared_diff = [] 
    for item in dataset[user1]:
        if item in dataset[user2]:
            squared_diff.append(np.square(dataset[user1][item] - dataset[user2][item]))
        
		# 이를 통해 유클리드 점수 계산
    return 1 / (1 + np.sqrt(np.sum(squared_diff))) 


# 사용자 1과 사용자 2 간의 피어슨 상관관계 점수 계산
def pearson_score(dataset, user1, user2):\
		# 마찬가지로 사용자가 없다면 오류 발생
    if user1 not in dataset:
        raise TypeError('Cannot find ' + user1 + ' in the dataset')

    if user2 not in dataset:
        raise TypeError('Cannot find ' + user2 + ' in the dataset')

    # 사용자 1과 사용자 2가 평가한 영화 추적 
    common_movies = {}

    for item in dataset[user1]:
        if item in dataset[user2]:
            common_movies[item] = 1

    num_ratings = len(common_movies) 

    # 공통으로 평가한 영화가 없다면 유사성 점수 = 0
    if num_ratings == 0:
        return 0

    # 두 사용자 모두가 평가한 영화의 평점의 합을 각각 계산
    user1_sum = np.sum([dataset[user1][item] for item in common_movies])
    user2_sum = np.sum([dataset[user2][item] for item in common_movies])

    # 마찬가지로 공통된 영화의 평점 제곱의 합을 각각 계산
    user1_squared_sum = np.sum([np.square(dataset[user1][item]) for item in common_movies])
    user2_squared_sum = np.sum([np.square(dataset[user2][item]) for item in common_movies])

    # 공통 영화에 대한 두 유저의 평점 곱의 합을 계산
    sum_of_products = np.sum([dataset[user1][item] * dataset[user2][item] for item in common_movies])

    # 위의 값들을 통해 피어슨 상관관계 점수 계산
    Sxy = sum_of_products - (user1_sum * user2_sum / num_ratings)
    Sxx = user1_squared_sum - np.square(user1_sum) / num_ratings
    Syy = user2_squared_sum - np.square(user2_sum) / num_ratings
    
		# 편차가 없다면 점수 = 0
    if Sxx * Syy == 0:
        return 0
		
		# 피어슨 점수 반환
    return Sxy / np.sqrt(Sxx * Syy)


# main 함수 정의 및 입력 매개변수 파싱
if __name__=='__main__':
    # args = build_arg_parser().parse_args()
    # user1 = args.user1
    # user2 = args.user2
    # score_type = args.score_type
    
    user1 = "David Smith"
    user2 = "Bill Duffy"
    score_type = "Euclidean"

		# 평가 등급을 딕셔너리로 읽어옴 
    ratings_file = 'ratings.json'

    with open(ratings_file, 'r') as f:
        data = json.loads(f.read())
	
    print("\nEuclidean score:")
    print(euclidean_score(data, user1, user2))
    print("\nPearson score:")
    print(pearson_score(data, user1, user2))


Euclidean score:
0.585786437626905

Pearson score:
0.9909924304103233
